# Imports

In [ ]:
import pandas as pd
from collections import defaultdict
import os
import math

# Load the data

In [ ]:
PATH_TO_FILE = #the file with all the data (accidents, involved, injured, schools, etc...)

In [ ]:
total_df = pd.read_csv(PATH_TO_FILE)

## Add involved unique ID

In [ ]:
total_df['inv_unique_id'] = total_df['provider_and_id'].astype(str) + '_' +  total_df['involve_id'].astype(str) + '_' + total_df['accident_year'].astype(str)


## Add accident unique ID

In [ ]:
total_df['accident_unique_id'] = total_df['provider_and_id'].astype(str) + '_' +  total_df['accident_year'].astype(str)

## Add 'vehicle_or_pedastrian' field

In [ ]:
total_df['vehicle_or_pedastrian'] = total_df.apply(lambda x: x['involve_vehicle_type_hebrew'] if x['injured_type_hebrew'] != 'הולך רגל' else  x['injured_type_hebrew'],
                                                  axis=1)


#### check vehicle_or_pedastrian

In [ ]:
total_df['vehicle_or_pedastrian'].unique()

In [ ]:
total_df.shape

### count schools

In [ ]:
total_df.school_id.nunique()

# Generate accidents per city files

In [ ]:
cities = [
    "מודיעין עילית",
    "אבו גוש",
    "אור יהודה",
    "בחן",
    "בית שמש",
    "ביתר עילית",
    "בן שמן (כפר נו",
    "הוד השרון",
    "חגי",
    "חדרה",
    "חולון",
    "ירושלים",
    "נצרת עילית",
    "נתניה",
    "קרית שמונה",
    "שריד",
    "תל אביב - יפו",
    "תראבין א-צאנע(",
    "אשדוד",
    "הרצליה",
    "חיפה",
    "חשמונאים",
    "טירה",
    "כפר ביאליק",
    "לוד",
    "מתתיהו",
    "קרית ביאליק",
    "ראשון לציון",
    "רחובות",
    "רמלה",
    "באר שבע",
    "פתח תקווה",
    "רמת גן",
    "אלעד",
    "בני ברק"
]

In [ ]:
CITIES_DIRECTORY = # a directory in which all the folders representing cities are saved

In [ ]:
for city in cities:
    print(f"שמירת כל התאונות בעיר {city}")
    city_data = total_df[total_df.school_yishuv_name == city]
    city_path = f"{CITIES_DIRECTORY}/{city}"
    if not os.path.exists(city_path):
        os.makedirs(city_path)
    file_path = f"{city_path}/תאונות בסביבת בתי הספר.csv"
    city_data.to_csv(file_path, index=False, encoding='utf-8-sig')

# Get injuries summary for each city

In [ ]:
KILLED_WEIGHT = 8
SEVERE_WEIGHT = 5
LIGHT_INJURED_WEIGHT = 1

In [ ]:
PRAT_KILLED_WEIGHT = 6600/7581
PRAT_SEVERE_WEIGHT = 956/7581
PRAT_LIGHT_WEIGHT = 25/7581

In [ ]:
def get_injuries_summary(key: str, val: str) -> dict:
    relevant_data = total_df[getattr(total_df, key) == val]
    killed = relevant_data.loc[relevant_data.injury_severity_hebrew == 'הרוג'].inv_unique_id.nunique()
    severe_injured = relevant_data.loc[relevant_data.injury_severity_hebrew == 'פצוע קשה'].inv_unique_id.nunique()
    light_injured = relevant_data.loc[relevant_data.injury_severity_hebrew == 'פצוע קל'].inv_unique_id.nunique()
    total_accidents = relevant_data.accident_unique_id.nunique()
    prat_score = (killed * PRAT_KILLED_WEIGHT + severe_injured * PRAT_SEVERE_WEIGHT  + light_injured * PRAT_LIGHT_WEIGHT) * (total_accidents)
    heuristic_score = killed * KILLED_WEIGHT + severe_injured * SEVERE_WEIGHT + light_injured * LIGHT_INJURED_WEIGHT
    return {
        'פצועים קל': light_injured,
        'פצועים קשה': severe_injured,
        'הרוגים': killed,
        'מספר התאונות': total_accidents,
        'ציון פר"ת': round(prat_score, 4),
        'ציון יוריסטי': round(heuristic_score, 4)
    }

In [ ]:
cities_injuries_summary = [{
    'שם העיר': city,
    **get_injuries_summary(key='school_yishuv_name', val=city)
    } for city in cities]
injuries_summary_df = pd.DataFrame(cities_injuries_summary)
file_path = f"{CITIES_DIRECTORY}/נפגעים בערים.csv"
injuries_summary_df.to_csv(file_path, index=False, encoding='utf-8-sig')

### Get injuries summary per school

In [ ]:
def get_bounding_box(latitude, longitude, distance_in_km):
    latitude = math.radians(latitude)
    longitude = math.radians(longitude)

    radius = 6371
    # Radius of the parallel at given latitude
    parallel_radius = radius * math.cos(latitude)

    lat_min = latitude - distance_in_km / radius
    lat_max = latitude + distance_in_km / radius
    lon_min = longitude - distance_in_km / parallel_radius
    lon_max = longitude + distance_in_km / parallel_radius
    rad2deg = math.degrees

    return rad2deg(lat_min), rad2deg(lon_min), rad2deg(lat_max), rad2deg(lon_max)

In [ ]:
for city in cities:
    print(f"חישוב פצועים בעיר {city}")
    city_schools = []
    city_schools_ids = total_df[total_df.school_yishuv_name == city].school_id.unique()
    for school_id in city_schools_ids:
        school_data = total_df[total_df.school_id == school_id]
        # using the last row to get the most updated data
        school_name = school_data.iloc[-1].school_name
        school_long, school_lat = school_data.iloc[-1].longitude, \
            school_data.iloc[-1].latitude
        lat_min, lon_min, lat_max, lon_max = get_bounding_box(latitude=school_lat, longitude=school_long,
        distance_in_km=0.5)
        baseX = lon_min
        baseY = lat_min
        distanceX = lon_max
        distanceY = lat_max
        school_json = {
            'סמל בית הספר': school_id,
            'שם בית הספר': school_name,
            **get_injuries_summary(key='school_id', val=school_id),
            '(x1, y1)': (baseX, baseY),
            '(x1, y2)': (baseX, distanceY),
            '(x2, y2)': (distanceX, distanceY),
            '(x2, y1)': (distanceX, baseY)
        }
        city_schools.append(school_json)
    city_path = f"{CITIES_DIRECTORY}/{city}"
    if not os.path.exists(city_path):
        os.makedirs(city_path)
    file_path = f"{city_path}/נפגעים בבתי הספר.csv"
    pd.DataFrame(city_schools).to_csv(file_path, index=False, encoding='utf-8-sig')

### Get injured with POINT(longitude, latitude)

In [ ]:
injured_with_points = total_df.copy()

In [ ]:
injured_with_points['WKT'] = injured_with_points.apply(lambda row: f'POINT ({row.longitude} {row.latitude})', axis=1)

In [ ]:
injured_with_points = injured_with_points[['WKT', 'vehicle_or_pedastrian', 'injury_severity_hebrew', 'age_group_hebrew', 'longitude', 'latitude', 'accident_timestamp']]

In [ ]:
# otherwise excel auto-detects this column as "Date" column
injured_with_points.age_group_hebrew = injured_with_points.age_group_hebrew.apply(lambda val: f' {val}')

In [ ]:
injured_with_points.rename(columns={
    'vehicle_or_pedastrian': 'סוג נפגע',
    'injury_severity_hebrew': 'חומרת פגיעה',
    'age_group_hebrew': 'קבוצת גיל',
    'accident_timestamp': 'תאריך התאונה'
}, inplace=True)

In [ ]:
injured_with_points.to_csv('{YOUR_PATH}/נפגעים עם מיקום.csv',
index=False, encoding='utf-8-sig')

### Get schools with polygons

In [ ]:
ids = total_df.school_id.unique()

In [ ]:
schools_with_polygons = []

for id_ in ids:
    school_data = total_df[total_df.school_id == id_]
    school_record = school_data.iloc[-1] # get the last record of the school
    # to get the most updated data
    school_long, school_lat = school_record.school_longitude, school_record.school_latitude
    lat_min, lon_min, lat_max, lon_max = get_bounding_box(latitude=school_lat, longitude=school_long, 
    distance_in_km=0.5)
    baseX = lon_min
    baseY = lat_min
    distanceX = lon_max
    distanceY = lat_max
    pol_str = "POLYGON (({0} {1}, {0} {3}, {2} {3}, {2} {1}, {0} {1}))".format(
        baseX, baseY, distanceX, distanceY
    )
    schools_with_polygons.append({
      'WKT': pol_str,
      'שם מוסד': school_record.school_name,
      'ישוב': school_record.school_yishuv_name,
      'longitude': school_long,
      'latitude': school_lat
    })

schools_with_polygons_df = pd.DataFrame(schools_with_polygons)

In [ ]:
schools_with_polygons_df.to_csv('{YOUR_PATH}/בתי ספר עם פוליגונים.csv',
index=False, encoding='utf-8-sig')